In [34]:
import csv
import ipyparallel as ipp
import timeit
import sys
from itertools import groupby

In [13]:
with open('/global/home/users/pierrj/testfiles/500.testbedtools.processed', newline = '') as eccloc:
    eccloc_reader = csv.reader(eccloc, delimiter = '\t')
    eccloc_list = [[int(row[0][10:12]), int(row[1]), int(row[2])] for row in eccloc_reader]
    
with open('/global/home/users/pierrj/testfiles/500.testbedtools.processed', newline = '') as eccloc:
    eccloc_reader = csv.reader(eccloc, delimiter = '\t')
    eccloc_list_forindex = [[int(row[0][10:12]) - 1, int(row[1]), int(row[2])] for row in eccloc_reader]
    
with open('/global/home/users/pierrj/testfiles/sorted.discordantmappedreads.oppositefacing.bed', newline = '') as discordant:
    discordant_reader = csv.reader(discordant, delimiter = '\t')
    discordant_list = [[int(row[0][10:12]), int(row[1]), int(row[2])] for row in discordant_reader]

In [5]:
discordant_indexed = [[x[1:] for x in g] for k, g in groupby(discordant_list, key = lambda x: x[0])]

In [45]:
start_time = timeit.default_timer()
with open('/global/home/users/pierrj/testfiles/sorted.discordantmappedreads.oppositefacing.bed', newline = '') as discordant:
    discordant_reader = csv.reader(discordant, delimiter = '\t')
    discordant_list = [[int(row[0][10:12]), int(row[1]), int(row[2])] for row in discordant_reader]
    discordant_indexed = [[x[1:] for x in g] for k, g in groupby(discordant_list, key = lambda x: x[0])]
print(timeit.default_timer() - start_time)

start_time = timeit.default_timer()
with open('/global/home/users/pierrj/testfiles/discordantmappedreads.oppositefacing.bed') as discordant:
    discordant_reader = csv.reader(discordant, delimiter = '\t')
    discordant_indexed_biglist = [[] for i in range(56)]
    for row in discordant_reader:
        discordant_indexed_biglist[(int(row[0][10:12])-1)].append([int(row[1]), int(row[2])])
print(timeit.default_timer() - start_time)

discordant_indexed == discordant_indexed_biglist

19.61608135700226
18.827782221138477


True

In [40]:
print(discordant_indexed[55])

[[26, 177], [491, 642], [1384, 1535], [918, 1052], [942, 1087], [490, 641], [963, 1114], [1393, 1521], [253, 404], [58, 195], [866, 1017], [597, 748], [6, 157], [266, 417], [1404, 1555], [1241, 1392], [418, 569], [851, 1002], [69, 220], [514, 665], [386, 533], [1304, 1455], [1081, 1232], [870, 1008], [371, 497], [42, 193], [66, 217], [321, 472], [543, 694], [374, 520], [942, 1093], [577, 728], [17, 168], [890, 1041], [6, 157], [891, 1042], [1462, 1594], [1154, 1305], [1130, 1279], [951, 1102], [996, 1147], [866, 988], [865, 1016], [377, 528], [1370, 1521], [1135, 1286], [1063, 1214], [1348, 1488], [854, 1002], [1026, 1163], [382, 533], [1314, 1465], [618, 769], [908, 1059], [1199, 1350], [1478, 1629], [1095, 1219], [1351, 1502], [469, 620], [1197, 1324], [1015, 1129], [852, 1003], [1118, 1240], [961, 1112], [1071, 1222], [1346, 1497], [1083, 1234], [937, 1080], [307, 458], [560, 711], [1331, 1482], [1524, 1649], [1451, 1602], [1135, 1286], [888, 1039], [432, 583], [440, 591], [996, 114

In [37]:
def get_size(obj, seen=None):

    """Recursively finds size of objects"""

    size = sys.getsizeof(obj)

    if seen is None:

        seen = set()

    obj_id = id(obj)

    if obj_id in seen:

        return 0

    # Important mark as seen *before* entering recursion to gracefully handle

    # self-referential objects

    seen.add(obj_id)

    if isinstance(obj, dict):

        size += sum([get_size(v, seen) for v in obj.values()])

        size += sum([get_size(k, seen) for k in obj.keys()])

    elif hasattr(obj, '__dict__'):

        size += get_size(obj.__dict__, seen)

    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):

        size += sum([get_size(i, seen) for i in obj])

    return size

In [6]:
def confirmeccs_noindex(ecc):
    for i in range(0, len(discordant_list), 2):
        read1 = discordant_list[i]
        read2 = discordant_list[i+1]
        if read1[0]==ecc[0]==read2[0]:
            if ecc[1] <= read1[1] <= ecc[2] and ecc[1] <= read1[2] <= ecc[2] and ecc[1] <= read2[1] <= ecc[2] and ecc[1] <= read2[2] <= ecc[2]:
                return True
    return False

In [21]:
def confirmeccs(ecc):
    for i in range(0, len(discordant_indexed[ecc[0]]), 2):
        read1 = discordant_indexed[ecc[0]][i]
        read2 = discordant_indexed[ecc[0]][i+1]
        if ecc[1] <= read1[0] <= ecc[2] and ecc[1] <= read1[1] <= ecc[2] and ecc[1] <= read2[0] <= ecc[2] and ecc[1] <= read2[1] <= ecc[2]:
            return True
    return False

In [22]:
yesornoeccs = list(map(confirmeccs, eccloc_list_forindex))

In [17]:
yesornoeccs_noindex = list(map(confirmeccs_noindex, eccloc_list))

In [23]:
yesornoeccs == yesornoeccs_noindex

True

In [26]:
from itertools import compress

In [27]:
confirmedeccs = list(compress(eccloc_list_forindex, yesornoeccs))

In [28]:
print(confirmedeccs)

[[14, 70735, 71303], [14, 349128, 349764], [2, 1674894, 1680235], [12, 355104, 356042], [2, 626071, 627316], [21, 209465, 209961], [4, 2696604, 2699034], [14, 132347, 132906], [9, 932917, 935653], [5, 1887602, 1889159], [8, 166497, 166880], [4, 3034338, 3034647], [1, 4945547, 4946469], [2, 573874, 575341], [11, 748313, 748925], [2, 3611761, 3616320], [7, 1295634, 1297723], [2, 3923725, 3924836], [4, 1884379, 1885179], [3, 1200621, 1200838], [8, 1087179, 1087434], [3, 3985805, 3986558], [12, 284307, 284726], [1, 2723079, 2723598], [4, 514531, 514923], [5, 1798055, 1799288], [0, 2572624, 2601296], [5, 1639975, 1640643], [2, 42591, 969033], [2, 969033, 974891], [6, 301843, 302179], [1, 3651347, 3651857], [1, 2975018, 2975126], [0, 2766710, 2767281], [1, 4781704, 4783066], [4, 903719, 904716], [3, 2745821, 2747117], [3, 1922629, 1923069], [1, 3122436, 3122929], [18, 16213, 17324], [2, 2233907, 2234838], [0, 1856983, 1857333], [13, 629643, 630109], [4, 2329237, 2331733], [6, 1706117, 170665

In [25]:
start_time = timeit.default_timer()
yesornoeccs = list(map(confirmeccs, eccloc_list_forindex))
print(timeit.default_timer() - start_time)

start_time = timeit.default_timer()
yesornoeccs_noindex = list(map(confirmeccs_noindex, eccloc_list))
print(timeit.default_timer() - start_time)

5.66011764574796
83.16817372618243


In [29]:
with open('/global/home/users/pierrj/testfiles/500.confirmed', 'w', newline = '') as confirmed:
    w = csv.writer(confirmed, delimiter = '\t')
    w.writerows(confirmedeccs)